# Modelos probabilísticos (Ejercicio)

## Aplicación de Naive Bayes multinomial a la detección de SMS *spam*

En este ejercicio se pide reproducir lo realizado en el caso práctico que se ha descrito en los vídeos (análisis de sentimiento en críticas de cine), pero ahora para detectar cuándo un mensaje corto (SMS) es *spam*.

### El conjunto de datos

El conjunto de datos consiste una serie de mensajes SMS (5574 en total), que están clasificados como mensajes basura (*spam*) o mensajes normales (*ham*). Los datos se pueden obtener en el [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection). 

En concreto, descargar el fichero [smsspamcollection.zip](https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip), y descomprimirlo para obtener un fichero de texto SMSSpamCollection. En este fichero de texto hay una línea por cada sms, con el formato: *clase* *tabulador* *sms*. Por ejemplo, la primera línea es:

`ham	Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...`

El fichero debe ser leído convenientemente para poder aplicar la vectorización. Se puede hacer la lectura usando las funciones python de lectura de ficheros, pero se recomienda usar la instrucción `read_table` de la biblioteca `pandas`:

In [339]:
import pandas as pd

*Pandas* es una biblioteca de python muy utilizada para manipular y analizar datos. Si el fichero se lee con la orden `read_table` (se pide averiguar la manera concreta de hacerlo), entonces se obtendrá una tabla (o *Data Frame*), en el que las etiquetas serán una columna y los correspondientes sms otra. Esto permite obtener de manera sencilla la lista de etiquetas o clases, y por otro lado la lista de mensajes, en el mismo orden.  

### Aprendiendo a clasificar SMSs

Se pide reproducir con estos datos lo realizado en el *notebook* en el que se aplica Naive Bayes Multinomial al análisis de sentimientos de críticas de cine, pero ahora para clasificar un SMS como *spam* o como normal. Esto incluye:

* Separación de los textos en entrenamiento y prueba 
* Vectorización de los textos 
* Aprendizaje con `MultinomialNB`
* Mostrar algunas clasificaciones sobre sms concretos.
* Rendimiento sobre entrenamiento y prueba.
* Ajuste manual del parámetro de suavizado
* Vectorización con `min_df` y `stop_words` 

**Nota**: este conjunto de datos no es balanceado (la mayoría son *ham*). Por tanto, usar `score` no es muy ilustrativo del rendimiento, ya que un clasificador "tonto" que siempre predijera *ham* tendría un rendimiento alto. Por ello, en este caso también se hace necesario usar el método `confusion_matrix` del módulo `metrics`. Se pide también explicar la salida que proporciona dicha métrica.

Se pide **comentar adecuadamente cada paso realizado**, relacionándolo con lo visto en la teoría. En particular, se pide mostrar parte de los atributos `class_count_`, `class_log_prior_`, `feature_count_` y `feature_log_prob_`, explicando claramente qué son cada uno de ellos. Explicar también cómo realiza las predicciones el modelo aprendido, tal y como se ha explicado en la teoría.  



## Ejercicio

Para leer el fichero que contiene los sms se ha utilizado la biblioteca recomendada `pandas`, en concreto, la instrucción `read_table`. Como el fichero donde se encuentran los sms tiene el formato *clase* *tabulador* *sms*, es necesario hacer unas modificaciones para que facilite la lectura y manejo de este. Para ello se ha hecho uso de una serie de parámetros que permite el método *read_table*:
- Primer parámetro es la ubicación donde se encuentra el fichero que queremos leer.
- *header = None* para indicar que el fichero no tiene cabecera. Si no se indica este valor, utilizará la primera línea como cabecera.
- *names* es el nombre que queremos darle a las cabeceras, para luego poder acceder mediante estos a las columnas correspondientes
- *converters* ya que tendremos que trabajar con valores de clasificación binarios (0 y 1) y en el fichero esta clasificación viene como texto ('spam' y 'ham'), mediante el parámetro converters indicamos que a los valores de la columna 'tag' en caso de ser 'ham' los cambie al valor 1 y en caso contrario a 0

Este método tiene otro parámetro, opcional, llamado *sep* para indicar el delimitador. En el caso del fichero con el que vamos a trabajar, el delimitador sería *tabulador* que es el mismo valor por defecto del parámetro *sep*, por eso en este caso no es necesario indicarlo

In [340]:
# Pandas
convertTag = lambda x: 1 if x == 'ham' else 0
collection = pd.read_table('smsspamcollection/SMSSpamCollection', header=None, names = ['tag','text'], converters={'tag': convertTag } )
target_names = ['spam', 'ham']
messages = collection.text.tolist()
tags = collection.tag.tolist() #y_train

Ahora que hemos leido el fichero, podemos acceder a los datos por sus columnas. Por ejemplo, veamos el quinto sms:

In [341]:
print("messages[4]: {}".format(messages[4]))
print("tags[4]: {}".format(tags[4]))
print("Clasificación asociada: {}".format(target_names[tags[4]]))

messages[4]: Nah I don't think he goes to usf, he lives around here though
tags[4]: 1
Clasificación asociada: ham


Como en el conjunto de datos no tenemos ejemplos de prueba, vamos a dividir la colección en un conjunto de datos de entrenamiento (80%) y otro conjunto de datos de pruebas (20%). Para ellos utilizamos el método `train_test_split` visto anteriormente en la asignatura.

In [342]:
# Separacion de los textos en entrenamiento y prueba
from sklearn.model_selection import train_test_split
text_train, text_test, y_train, y_test = train_test_split(messages, tags, stratify = tags, test_size = 0.2)

Para comprobar si es un conjunto balanceado hacemos uso del método `bincount` de *numpy*. Vamos a ver tanto del conjunto original, como de los nuevos conjuntos de entrenamiento y prueba. Y comprobamos que no está balanceado, pues hay una gran diferencia entre el número de mensajes clasificados como 'spam' (747) y los clasificados como 'ham' (4825)

In [343]:
import numpy as np
print("Ejemplos por cada clase conjunto original: {}".format(np.bincount(tags)))
print("Ejemplos por cada clase conjunto de entrenamiento (train): {}".format(np.bincount(y_train)))
print("Ejemplos por cada clase prueba (test): {}".format(np.bincount(y_test)))

Ejemplos por cada clase conjunto original: [ 747 4825]
Ejemplos por cada clase conjunto de entrenamiento (train): [ 598 3859]
Ejemplos por cada clase prueba (test): [149 966]


Para poder aplicar Naive Bayes a un texto, hay que representar los documentos mediante vectores numéricos. Para ello vamos a fijar los términos del vocabulario (palabras que aparecen en nuestro *corpus*) mediante un vector en el cual en cada componente tenemos el número de veces que aparece cada término en el vocabulario. Con esto estamos dando relevancia al término por el número de veces que se utiliza, ignorando el orden en el que aparecen. <br>
Mediante `CountVectorizer` hacemos esta representación vectorial y con el método *fit*, dado los documentos, obtenemos el vocabulario en en una matriz dispersa.<br>
Una vez tenemos el vocabulario, transformamos los documentos en vectores (`transform`), tanto los del conjunto entrenamiento como los del conjunto de pruebas.

In [344]:
# Vectorizacion de los textos
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer().fit(messages)
print("Tamaño del vocabulario: {}".format(len(vect.vocabulary_)))
# print("Vocabulario:\n {}".format(vect.vocabulary_))

X_train = vect.transform(text_train)
X_test = vect.transform(text_test)

Tamaño del vocabulario: 8713


Ahora podemos aplicar `Naive Bayes Multinomial` para poder detectar los SMS que sean spam. Con el método *fit* se entrena al modelo, es decir, se cuentan los términos por cada clase y se calcula las log-probabilidades.

In [345]:
# Aprendizaje con `MultinomialNB`
from sklearn.naive_bayes import MultinomialNB
multinb=MultinomialNB().fit(X_train,y_train)

A continuación vamos a ver una serie de atributos que nos proporciona este entrenamiento y que son interesantes:
- *class_count_* : indica el número de ejemplo encontrado de cada clase. Para el conjunto de entrenamiento que se ha utilizado, nos indica que hay 598 con clasificación 'spam' y 3859 'ham'. Volvemos a confirmar que no está balanceado.
- *class_log_prior_* : las log-probabilidades (para evitar problemas de número bajos) para cada clase.
- *feature_count_* : número de veces que se repite cada término en cada una de las clasificaciones. Es un array que contiene a su vez 2 arrays, uno por cada clasificación.
- *feature_log_prob_* : las log-probabilidades de las ocurrencias de cada término para una clase respecto a todos los términos en cada clase

In [346]:
print(multinb.class_count_)
print(multinb.class_log_prior_)
print(multinb.feature_count_)
print(multinb.feature_log_prob_)

[ 598. 3859.]
[-2.00864042 -0.14406781]
[[10. 22.  0. ...  0.  1.  0.]
 [ 0.  0.  1. ...  1.  0.  1.]]
[[ -7.63495249  -6.89735354 -10.03284776 ... -10.03284776  -9.33970058
  -10.03284776]
 [-10.99231883 -10.99231883 -10.29917165 ... -10.29917165 -10.99231883
  -10.29917165]]


Ahora que ha sido entrenado, vamos a probar con 2 ejemplos cogidos al azar para comparar la clasificación real y la clasificación que predice el clasificador. Primero vemos los mensajes y sus clasificaciones correspondientes de los mensajes octavo y décimo del conjunto de pruebas.<br>
A continuación la predicción de las clasificaciones de cada uno de ellos y la seguridad que tenía en cada predicción (`predict_proba`). <br>
En estos casos concretos ha acertado la predicción con lo clasificado, aunque puede darse lo contrario y no acertar.

In [347]:
# Clasificaciones de sms concretos
print("Octavo mensaje del conjunto de test:\n{}".format(text_test[7]))
print("Clasificación verdadera: {}.\n".format(y_test[7]))

print("Décimo mensaje del conjunto de test:\n{}".format(text_test[21]))
print("Clasificación verdadera: {}\n".format(y_test[9]))

print("Predicción del clasificador para el noveno mensaje: {}".format(multinb.predict(vect.transform([text_test[7]]))[0]))
print("Predicción del clasificador para el décimo mensaje: {}\n".format(multinb.predict(vect.transform([text_test[9]]))[0]))

print("Predicción de probabilidad para el noveno mensaje: {}".format(multinb.predict_proba(vect.transform([text_test[7]]))[0]))
print("Predicción de probabilidad para el décimo mensaje: {}\n".format(multinb.predict_proba(vect.transform([text_test[9]]))[0]))

Octavo mensaje del conjunto de test:
Yay can't wait to party together!
Clasificación verdadera: 1.

Décimo mensaje del conjunto de test:
Congratulations ur awarded either £500 of CD gift vouchers & Free entry 2 our £100 weekly draw txt MUSIC to 87066 TnCs www.Ldew.com 1 win150ppmx3age16
Clasificación verdadera: 1

Predicción del clasificador para el noveno mensaje: 1
Predicción del clasificador para el décimo mensaje: 1

Predicción de probabilidad para el noveno mensaje: [9.37247706e-05 9.99906275e-01]
Predicción de probabilidad para el décimo mensaje: [1.36233472e-04 9.99863767e-01]



Como se ha comentado anteriormente, al no ser un conjunto balanceado no tiene mucho sentido ver el rendimiento haciendo uso del método *score*. En su lugar vamos a utilizar el método `confusion_matrix` que nos evalúa la precisión de una clasificación. Este método recibe las clasificaciones reales y las predicciones, y obtenemos como resultado una matriz el número de positivos negativos, falsos negativos, verdaderos positivos y falsos positivos.<br>
Con este resultado vamos a ver el rendimiento mediante la fórmula de probabilidad vista en la teoría:<br>
$$ P (A \mid B) = \frac{P (B \mid A) P (A)}{P (B)} $$
La aplicamos al conjunto de entrenamiento y al de pruebas.

In [348]:
# Rendimiento sobre entrenamiento y prueba.
predictTrain = multinb.predict(X_train)
predictTest = multinb.predict(X_test)
from sklearn.metrics import confusion_matrix
matrixTrain = confusion_matrix(y_train, predictTrain)
spamsTrain = matrixTrain[0][0]/(matrixTrain[0][0]+matrixTrain[0][1])
hamsTrain = matrixTrain[1][1]/(matrixTrain[1][1]+matrixTrain[1][0])
print('Rendimiento del conjunto de entrenamiento')
print('Matriz de confusión')
print(matrixTrain)
print("Aciertos de la clasificación de 'spam': {}".format(round(spamsTrain, 3)))
print("Aciertos de la clasificación de 'ham': {}\n".format(round(hamsTrain,3)))

matrixTest = confusion_matrix(y_test, predictTest)
spamsTest = matrixTest[0][0]/(matrixTest[0][0]+matrixTest[1][0])
hamsTest = matrixTest[1][1]/(matrixTest[1][1]+matrixTest[0][1])
print('Rendimiento del conjunto de prueba')
print('Matriz de confusión')
print(matrixTest)
print("Aciertos de la clasificación de 'spam': {}".format(round(spamsTest, 3)))
print("Aciertos de la clasificación de 'ham': {}".format(round(hamsTest, 3)))

Rendimiento del conjunto de entrenamiento
Matriz de confusión
[[ 581   17]
 [   9 3850]]
Aciertos de la clasificación de 'spam': 0.972
Aciertos de la clasificación de 'ham': 0.998

Rendimiento del conjunto de prueba
Matriz de confusión
[[140   9]
 [ 13 953]]
Aciertos de la clasificación de 'spam': 0.915
Aciertos de la clasificación de 'ham': 0.991


Viendo los porcentajes obtenidos podemos decir que el modelo clasifica bien, pues para ambas clasificaciones obtenemos un porcentaje alto (más del 90%)

En este algoritmo tambien podemos modificar el parámetro de suavizado (*alpha*), que por defecto es 1. Cuanto menos suavizado esté, es decir, cuanto menor sea el valor de *alpha*, más se ajustará al conjunto de entrenamiento, lo cual no es bueno cuando se pueden obtener probabilidades muy bajas.
Veamos los resultado poniendo un valor de *alpha* menor que 1, por ejemplo, 0.5

In [349]:
multinb_aplha_05=MultinomialNB(alpha=0.5).fit(X_train,y_train)
predictTrain_05 = multinb_aplha_05.predict(X_train)
matrixTrain_05 = confusion_matrix(y_train, predictTrain_05)
spamsTrain_05 = matrixTrain_05[0][0]/(matrixTrain_05[0][0]+matrixTrain_05[0][1])
hamsTrain_05 = matrixTrain_05[1][1]/(matrixTrain_05[1][1]+matrixTrain_05[1][0])
print('Rendimiento del conjunto de entrenamiento con alpha = 0.5')
print('Matriz de confusión')
print(matrixTrain_05)
print("Aciertos de la clasificación de 'spam': {}".format(round(spamsTrain_05, 3)))
print("Aciertos de la clasificación de 'ham': {}\n".format(round(hamsTrain_05,3)))

Rendimiento del conjunto de entrenamiento con alpha = 0.5
Matriz de confusión
[[ 587   11]
 [  17 3842]]
Aciertos de la clasificación de 'spam': 0.982
Aciertos de la clasificación de 'ham': 0.996



Y ahora veamos ocn un *aplha* mayor que 1, por ejemplo, 150

In [350]:
multinb_aplha_150=MultinomialNB(alpha=150).fit(X_train,y_train)
predictTrain_150 = multinb_aplha_150.predict(X_train)
matrixTrain_150 = confusion_matrix(y_train, predictTrain_150)
spamsTrain_150 = matrixTrain_150[0][0]/(matrixTrain_150[0][0]+matrixTrain_150[0][1])
hamsTrain_150 = matrixTrain_150[1][1]/(matrixTrain_150[1][1]+matrixTrain_150[1][0])
print('Rendimiento del conjunto de entrenamiento con alpha = 150')
print('Matriz de confusión')
print(matrixTrain_150)
print("Aciertos de la clasificación de 'spam': {}".format(round(spamsTrain_150, 3)))
print("Aciertos de la clasificación de 'ham': {}\n".format(round(hamsTrain_150,3)))

Rendimiento del conjunto de entrenamiento con alpha = 150
Matriz de confusión
[[  20  578]
 [   0 3859]]
Aciertos de la clasificación de 'spam': 0.033
Aciertos de la clasificación de 'ham': 1.0



Comparando los resultados obtenidos con los distintos alpha, podríamos decir que con alpha 150 se mejora mucho la clasificación para los mensajes 'ham', en cambio, la clasificación para los mensajes 'spam' empeora demasiado. Sin embargo, con un valor más bajo de alpha, en este caso el probrado con 0.5, se mantiene una buena clasificación de ambos mensajes, siendo mayor del 90%

Mediante el uso de `GridSearch` podemos indicarle distintos posibles valores de *alpha* para que encuentre cuál de ellos es el más adecuado.

In [351]:
# Ajuste manual del parámetro de suavizado
from sklearn.model_selection import GridSearchCV
param_grid_nb = {'alpha': [0.0001,0.001, 0.01,0.1, 1, 10,100,200]}
grid_nb = GridSearchCV(MultinomialNB(), param_grid_nb, cv=5)
grid_nb.fit(X_train, y_train)
print("Mejor parámetro: ", grid_nb.best_params_)
print("Rendimiento de MultonomialNB en validación cruzada, con el mejor parámetro: {:.3f}".format(grid_nb.best_score_))

Mejor parámetro:  {'alpha': 1}
Rendimiento de MultonomialNB en validación cruzada, con el mejor parámetro: 0.983


In [352]:
multinb_best=MultinomialNB(alpha=0.01).fit(X_train,y_train)
predictTrain_best = multinb_best.predict(X_train)
matrix_best = confusion_matrix(y_train, predictTrain_best)
spams_best = matrix_best[0][0]/(matrix_best[0][0]+matrix_best[0][1])
hams_best = matrix_best[1][1]/(matrix_best[1][1]+matrix_best[1][0])
print('Rendimiento del conjunto de entrenamiento con mejor parámetro alpha')
print('Matriz de confusión')
print(matrix_best)
print("Aciertos de la clasificación de 'spam': {}".format(round(spams_best, 3)))
print("Aciertos de la clasificación de 'ham': {}".format(round(hams_best,3)))

Rendimiento del conjunto de entrenamiento con mejor parámetro alpha
Matriz de confusión
[[ 592    6]
 [   9 3850]]
Aciertos de la clasificación de 'spam': 0.99
Aciertos de la clasificación de 'ham': 0.998


De entre los distintos *alpha* indicados, incluido entre ellos el valor 1 por defecto, el mejor parámetro de suavizado es 0.01

Para mejorar la clasificación podemos hacer uso de las siguientes opciones:
- *`stop words`*: palabras usadas tan frecuentemente que no aportan valor a la clasificación
- *`min_df`*: para indicar el número mínimo de veces que debe aparecer un término para tenerlo en cuenta. De esta forma ignoramos de nuestro vocabulario aquellos que no aparecen mucho y que por tanto, no aportan valor a la clasificación, así ganar eficiencia pues tendremos menos características

Veamos como disminuye el número de términos del vocabulario haciendo uso de estas opciones con los siguientes valores:
- stop_words = 'english'
- min_df = 20

In [353]:
# Vectorización con `min_df` y `stop_words`
vect_red = CountVectorizer(min_df=20, stop_words="english").fit(text_train)
X2_train = vect_red.transform(text_train)
print("Número de términos en el vocabulario original: {}".format(len(vect.get_feature_names_out())))
feature_names2 = vect_red.get_feature_names_out()
print("Número de términos en el vocabulario con stop words y min_df: {}".format(len(feature_names2)))

Número de términos en el vocabulario original: 8713
Número de términos en el vocabulario con stop words y min_df: 328


Al reducir el vocabulario mejora ligeramente la clasificación de 'ham' y empeora levemente la clasificación de 'spam'

In [354]:
multinb_red=MultinomialNB().fit(X2_train,y_train)
predictTrain_min = multinb_red.predict(X2_train)
matrix_min = confusion_matrix(y_train, predictTrain_min)
spams_min = matrix_min[0][0]/(matrix_min[0][0]+matrix_min[0][1])
hams_min = matrix_min[1][1]/(matrix_min[1][1]+matrix_min[1][0])
print('Rendimiento del conjunto de entrenamiento con el vocabulario reducido')
print('Matriz de confusión')
print(matrix_min)
print("Aciertos de la clasificación de 'spam': {}".format(round(spams_min, 3)))
print("Aciertos de la clasificación de 'ham': {}\n".format(round(hams_min,3)))

Rendimiento del conjunto de entrenamiento con el vocabulario reducido
Matriz de confusión
[[ 548   50]
 [  45 3814]]
Aciertos de la clasificación de 'spam': 0.916
Aciertos de la clasificación de 'ham': 0.988

